In [2]:
import os
from glob import glob # 파일 경로 패턴 매칭에 유용

from dotenv import load_dotenv
load_dotenv()

True

## 1. LangChain RAG 구현

- RAG(Retrieval Augmented Generation)는 쉽게 말해, LLM이 **외부 데이터를 참고해서** 더 똑똑한 답변을 생성하도록 돕는 기술
- LLM의 고질적인 문제인 **환각 현상(Hallucination)을 줄이고**, **최신 정보나 특정 도메인 지식을 활용**할 수 있게 해주며, LLM한테 오픈북 시험 보게 하는 거랑 비슷함.

크게 다음과 같은 단계를 거침:
1.  **문서 로드(Load):** RAG의 재료가 될 외부 데이터(텍스트 파일, PDF, 웹페이지 등)를 불러옴.
2.  **분할(Split):** 불러온 문서를 LLM이 한 번에 처리하기 좋은 크기(청크)로 잘게 쪼개며, 너무 길면 LLM의 이슈 발생.
3.  **저장(Store):** 쪼갠 문서 조각(청크)들을 벡터 임베딩으로 변환해서 벡터 저장소에 넣어둠.
4.  **검색(Retrieve):** 사용자 질문이 들어오면, 질문과 의미적으로 유사한 문서 청크들을 벡터 저장소에서 찾아옴.
5.  **생성(Generate):** 찾아온 문서 청크들을 참고자료 삼아 LLM이 최종 답변을 생성함. 이때 질문과 참고자료를 LLM에게 잘 전달하는 게 중요함.

### 1.1 문서 로더 (Document Loaders)

- 첫 단계는 RAG 시스템의 지식 창고가 될 문서들을 불러오는 거임. 
LangChain은 다양한 형식의 문서를 로드할 수 있는 여러 `DocumentLoader`를 제공함. 
(예: `TextLoader`, `PyPDFLoader`, `CSVLoader`, `UnstructuredURLLoader` 등)

**팁:**
- 디렉터리 자체 로더도 유용하게 쓰일것 같음.

In [2]:
# 'data' 폴더 안에 있는 모든 '_KR.txt' 파일을 찾음
txt_files = glob(os.path.join('./data', '*_KR.txt'))

print(txt_files)

['./data\\Rivian_KR.txt', './data\\Tesla_KR.txt']


In [3]:
# 첫 번째 텍스트 파일 내용 확인 (샘플 로드)
from langchain_community.document_loaders import TextLoader

# TextLoader는 지정된 텍스트 파일을 로드함.
loader = TextLoader(txt_files[0], encoding='utf-8') 
data_sample = loader.load() # load() 메서드는 Document 객체의 리스트를 반환함.

# 로드된 Document 객체의 개수 확인. TextLoader는 파일 하나당 하나의 Document 객체를 생성함.
len(data_sample) 

1

In [4]:
# data_sample의 타입 확인. Document 객체들이 담긴 리스트임.
type(data_sample)

list

In [5]:
# 로드된 Document 객체의 내용(page_content)과 메타데이터(metadata)를 살펴봄.
# data_sample[0]은 첫 번째 Document 객체를 의미함.
# .page_content 속성에는 파일의 실제 텍스트 내용이 담겨있음.
print(data_sample[0].page_content)

2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다.

리비안의 초기 프로젝트는 피터 스티븐스가 디자인한 2+2 시트 배열의 미드십 엔진 하이브리드 쿠페 스포츠카 R1(원래 이름은 아베라(Avera))이었습니다. 이 차량은 모듈식 캡슐 구조와 쉽게 교체 가능한 차체 패널을 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산을 시작할 것으로 예상되었습니다. 리비안은 디젤 하이브리드, 브라질 원메이크 시리즈를 위한 R1 GT라는 이름의 레이싱 버전, 4도어 세단, 크로스오버 등 다양한 버전을 검토했습니다. 2011년에 프로토타입 해치백이 공개되었지만, R1과의 연관성은 아직 불분명합니다.

2021년 10월, 리비안은 첫 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.


In [6]:
# .metadata 속성에는 문서의 출처(source) 같은 부가 정보가 딕셔너리 형태로 저장됨.
print(data_sample[0].metadata)

{'source': './data\\Rivian_KR.txt'}


In [7]:
# 이제 data 폴더에 있는 모든 텍스트 파일들을 로드함.
from tqdm import tqdm # 진행 상황을 보여주는 라이브러리, 데이터 많을 때 유용

all_documents = [] # 로드된 Document 객체들을 담을 리스트

for text_file in tqdm(txt_files): # tqdm으로 감싸면 반복문 진행 상태가 표시됨
    loader = TextLoader(text_file, encoding='utf-8')
    all_documents += loader.load() # 각 파일에서 로드된 Document 리스트를 기존 리스트에 추가

# 총 몇 개의 Document가 로드되었는지 확인. 파일 개수만큼 생성됨.
len(all_documents)

100%|██████████| 2/2 [00:00<00:00, 292.46it/s]


2

In [8]:
# 첫 번째 로드된 문서의 내용 확인 (리비안_EN.txt)
print(all_documents[0].page_content)

2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다.

리비안의 초기 프로젝트는 피터 스티븐스가 디자인한 2+2 시트 배열의 미드십 엔진 하이브리드 쿠페 스포츠카 R1(원래 이름은 아베라(Avera))이었습니다. 이 차량은 모듈식 캡슐 구조와 쉽게 교체 가능한 차체 패널을 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산을 시작할 것으로 예상되었습니다. 리비안은 디젤 하이브리드, 브라질 원메이크 시리즈를 위한 R1 GT라는 이름의 레이싱 버전, 4도어 세단, 크로스오버 등 다양한 버전을 검토했습니다. 2011년에 프로토타입 해치백이 공개되었지만, R1과의 연관성은 아직 불분명합니다.

2021년 10월, 리비안은 첫 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.


In [9]:
# 두 번째 로드된 문서의 내용 확인 (테슬라_EN.txt)
print(all_documents[1].page_content)

텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 머스크(Elon Musk)의 적극적인 참여를 받았습니다. 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.

2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년 대비 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 누적 글로벌 판매량은 4,962,975대를 넘어섰습니다. SMT 패키징(SMT Packaging)에 따르면, 테슬라의 2023년 판매량은 글로벌 전기 자동차 시장의 약 12.9%를 차지했습니다.


In [10]:
# 로드된 각 항목은 LangChain의 Document 객체임.
type(all_documents[0])

langchain_core.documents.base.Document

### 1.2 텍스트 분할기 (Text Splitters)

LLM은 한 번에 처리할 수 있는 텍스트 양(토큰 수)에 제한이 있음. 
그래서 긴 문서를 통째로 넣기보다는, 의미적으로 관련된 작은 조각(청크)으로 나눠서 처리하는 게 일반적임.
텍스트 분할기는 이 역할을 담당함. 어떻게 나누느냐에 따라 RAG 성능에 영향을 줄 수 있으니 중요함!

**주요 파라미터:**
- `chunk_size`: 나눌 청크의 최대 크기 (글자 수 기준). LLM의 컨텍스트 윈도우 크기와 데이터 특성을 고려해서 정해야 함.
- `chunk_overlap`: 청크끼리 겹치는 글자 수. 청크 경계에서 의미가 잘리는 걸 방지하고 문맥 연속성을 유지하는 데 도움을 줌. 너무 크면 중복 데이터가 많아지고, 작으면 문맥 파악이 어려울 수 있음.
- `separator`: 청크를 나눌 기준이 되는 구분자. 기본값은 `"\n\n"`(연속된 개행 문자, 보통 문단 구분)임. 데이터 특성에 맞춰 변경 가능. (예: 문장 단위로 나누고 싶다면 `.` 등)

**팁:**
- `CharacterTextSplitter`는 가장 기본적인 분할기임. 좀 더 똑똑하게 나누고 싶다면 `RecursiveCharacterTextSplitter`를 고려해볼 수 있음 (문단 -> 문장 -> 단어 순으로 계층적 분할 시도).
- `chunk_size`와 `chunk_overlap`은 실험을 통해 최적값을 찾는 게 좋음.

In [11]:
# 분할하기 전에 각 문서의 원본 글자 수를 확인해봄.
char_count = [len(doc.page_content) for doc in all_documents]
print(char_count)

[551, 530]


In [12]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=250,        # 청크 최대 크기를 250자로 설정
    chunk_overlap=50,      # 청크 간 겹치는 부분을 50자로 설정
    separator='\n\n',      # 문단("\n\n")을 기준으로 우선 분할 시도
    # length_function=len, # 청크 길이를 계산하는 함수 (기본값은 len)
    # is_separator_regex=False # separator가 정규표현식인지 여부 (기본값은 False)
)

# Document 객체 리스트를 받아 분할된 Document 청크 리스트를 반환함
splitted_texts = text_splitter.split_documents(all_documents)

print(f"생성된 텍스트 청크 수: {len(splitted_texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in splitted_texts)}")

# 출력에 종종 보이는 "Created a chunk of size ... which is longer than the specified ..." 경고는 
# separator로 나눈 덩어리가 chunk_size보다 클 때 발생함. 
# CharacterTextSplitter는 separator를 우선으로 고려하고, 그 다음 chunk_size를 맞추려고 하기 때문임.

Created a chunk of size 304, which is longer than the specified 250
Created a chunk of size 339, which is longer than the specified 250


생성된 텍스트 청크 수: 5
각 청크의 길이: [193, 304, 50, 339, 189]


**`CharacterTextSplitter`의 동작 방식 및 경고 메시지 이해:**

- 위 코드에서 `chunk_size`를 250으로 설정했지만, 실제 생성된 청크 중에는 421자, 521자처럼 485자를 넘는 것들이 있음.
- `CharacterTextSplitter`는 먼저 `separator` (여기서는 `\n\n`)를 기준으로 텍스트를 나눔.
- 이렇게 나눠진 덩어리가 `chunk_size`보다 크면, 그 덩어리 안에서 다시 `chunk_size`에 맞춰 자르려고 시도함.
- 하지만 `separator`로 나눈 가장 작은 단위 자체가 `chunk_size`를 초과하면, 어쩔 수 없이 `chunk_size`보다 큰 청크가 생성될 수 있음. (예: 한 문단 자체가 300자인데 `chunk_size`가 250이라면, 그 문단은 통째로 하나의 청크가 될 가능성이 높음)
- 이게 바로 `Created a chunk of size ... which is longer than the specified ...` 경고가 뜨는 이유임.

**Q. `separator`를 빈 문자열(`''`)로 하면 어떻게 될까?**
- `separator=''`로 설정하면, 각 문자를 기준으로 나누려고 함. 
- 이렇게 되면 `chunk_size`에 거의 정확히 맞춰서 잘리겠지만, 단어나 문장의 의미가 완전히 깨질 수 있음. (아래 코드에서 직접 확인 가능) 
- 대부분의 경우 권장하지 않는 방식임. 문맥 유지가 중요하기 때문!

In [13]:
# 첫 번째 청크 내용 확인
print("첫 번째 청크의 내용:", splitted_texts[0].page_content)

첫 번째 청크의 내용: 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다.


In [14]:
# 두 번째 청크 내용 확인
print("두 번째 청크의 내용:", splitted_texts[1].page_content)

두 번째 청크의 내용: 리비안의 초기 프로젝트는 피터 스티븐스가 디자인한 2+2 시트 배열의 미드십 엔진 하이브리드 쿠페 스포츠카 R1(원래 이름은 아베라(Avera))이었습니다. 이 차량은 모듈식 캡슐 구조와 쉽게 교체 가능한 차체 패널을 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산을 시작할 것으로 예상되었습니다. 리비안은 디젤 하이브리드, 브라질 원메이크 시리즈를 위한 R1 GT라는 이름의 레이싱 버전, 4도어 세단, 크로스오버 등 다양한 버전을 검토했습니다. 2011년에 프로토타입 해치백이 공개되었지만, R1과의 연관성은 아직 불분명합니다.


**참고: `separator=''` 로 설정하고 분할한 결과**

아래 코드를 실행해보면 `chunk_size`에 거의 정확히 맞춰 잘리지만, 단어 중간에 잘리는 등 문맥이 심하게 훼손되는 것을 볼 수 있음.

In [15]:
# text_splitter_no_sep = CharacterTextSplitter(
#     chunk_size=250,        # 청크 크기  
#     chunk_overlap=50,      # 청크 중 중복되는 부분 크기
#     separator='',          # 구분자 없음 (각 문자를 기준으로)
# )

# texts_no_sep = text_splitter_no_sep.split_documents(all_documents)
# print(f"(separator='') 생성된 텍스트 청크 수: {len(texts_no_sep)}")
# print(f"(separator='') 각 청크의 길이: {list(len(text.page_content) for text in texts_no_sep)}")

In [16]:
# # (separator='') 겹치는 부분 출력 (의미가 깨지는 것을 확인)
# print("(separator='') 첫 번째 청크의 최종 50글자:", texts_no_sep[0].page_content[-50:])
# print("(separator='') 두 번째 청크의 처음 50글자:", texts_no_sep[1].page_content[:50])

### 1.3 임베딩 모델 (Embeddings)

텍스트를 컴퓨터가 이해하고 비교할 수 있는 숫자 표현, 즉 벡터(vector)로 바꾸는 과정을 '임베딩'이라고 함. 
의미가 비슷한 단어나 문장은 벡터 공간에서 가까운 곳에 위치하게 됨. 
RAG에서는 이 임베딩을 사용해서 사용자 질문과 가장 관련 있는 문서 청크를 찾아냄.

LangChain은 OpenAI, HuggingFace 등 다양한 임베딩 모델을 지원함. 
여기서는 OpenAI의 `text-embedding-3-small` 모델을 사용.

**임베딩 모델 선택 시 고려사항:**
- **성능:** 얼마나 의미를 잘 파악해서 벡터로 표현하는가?
- **비용:** API 호출 비용 또는 자체 호스팅 비용.
- **속도:** 임베딩 생성 속도.
- **차원 수:** 생성되는 벡터의 차원. 차원이 높을수록 더 많은 정보를 담을 수 있지만, 계산량이 늘어날 수 있음. (예: `text-embedding-3-small`은 1536차원)

**장점 (OpenAI Embeddings):**
- 비교적 높은 성능의 임베딩을 쉽게 사용 가능.

**단점 (OpenAI Embeddings):**
- API 호출 비용 발생.
- 인터넷 연결 필요.
- 데이터가 OpenAI 서버로 전송됨 (민감 데이터의 경우 고려 필요).

In [17]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # OpenAI의 임베딩 모델 중 하나. 더 큰 모델도 있음 (예: text-embedding-3-large)
)

# 샘플 텍스트를 임베딩해봄
sample_text = "테슬라 창업자는 누구인가요?"
vector = embeddings_model.embed_query(sample_text) # 단일 텍스트 임베딩은 embed_query 사용
# 여러 텍스트(문서)를 한 번에 임베딩하려면 embed_documents 사용

print(f"임베딩 벡터의 차원: {len(vector)}")
# print(f"임베딩 벡터 (앞부분 일부): {vector[:5]}") # 실제 벡터 값은 매우 긴 숫자 리스트

임베딩 벡터의 차원: 1536


### 1.4 벡터 저장소 (Vector Stores)

임베딩된 텍스트 청크(벡터)들을 저장하고, 나중에 유사도 기반으로 빠르게 검색할 수 있게 해주는 데이터베이스임. 
LangChain은 Chroma, FAISS, Pinecone, Weaviate 등 다양한 벡터 저장소를 지원함.
여기서는 사용하기 간편한 인메모리(또는 로컬 디스크 기반) 벡터 저장소인 `Chroma`를 사용.

**Chroma 사용법:**
- `from_documents()`: 분할된 텍스트 청크(`splitted_texts`)와 임베딩 모델(`embeddings_model`)을 전달하면, 내부적으로 각 청크를 임베딩하고 Chroma DB에 저장함.
- `persist_directory`: 지정된 경로에 DB를 저장하여 나중에 재사용할 수 있게 함. 지정 안 하면 인메모리로만 동작.
- `collection_name`: DB 내에서 특정 데이터셋을 구분하는 이름.

**장점 (Chroma):**
- 설치 및 사용이 간편함. 로컬 환경에서 테스트하거나 소규모 프로젝트에 적합.
- 디스크에 저장하여 영속성 확보 가능.

**단점 (Chroma):**
- 대규모 데이터나 고성능 검색 환경에서는 전문적인 벡터 DB(예: Pinecone, Weaviate)보다 성능이 떨어질 수 있음.

In [18]:
# 벡터 저장소에 넣을 텍스트 청크 중 첫 번째 것을 다시 한번 확인.
# 각 청크는 Document 객체 형태를 유지하고 있음 (page_content와 metadata 가짐)
splitted_texts[0]

Document(metadata={'source': './data\\Rivian_KR.txt'}, page_content='2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다.')

In [19]:
from langchain_chroma import Chroma

# Chroma 벡터 저장소 생성
# splitted_texts ("분할된 Document 청크들")와 embeddings_model (임베딩 생성기)을 전달
vectorstore = Chroma.from_documents(
    documents=splitted_texts,      # 임베딩하고 저장할 Document 객체 리스트
    embedding=embeddings_model,    # 텍스트를 벡터로 변환할 임베딩 모델
    collection_name="chroma_test", # 벡터 컬렉션 이름 (나중에 불러올 때 사용 가능)
    persist_directory="./chroma_db", # 벡터 데이터를 저장할 디렉토리. 지정하면 디스크에 저장됨.
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 5


**벡터 저장소에서 유사도 검색하기:**

벡터 저장소의 핵심 기능은 특정 쿼리(질문)가 주어졌을 때, 저장된 벡터들과의 유사도를 계산하여 가장 관련 높은 문서(청크)들을 찾아주는 것임.
`similarity_search()` 메서드를 사용하면 됨.
- `query`: 검색할 질문 텍스트.
- `k`: 반환받을 유사 문서의 개수 (기본값은 4). 

이 메서드는 내부적으로 다음 단계를 수행함:
1.  입력된 `query`를 임베딩 모델을 사용해 벡터로 변환.
2.  변환된 쿼리 벡터와 벡터 저장소에 있는 모든 문서 청크 벡터들 간의 유사도(보통 코사인 유사도)를 계산.
3.  유사도가 높은 순으로 `k`개의 문서 청크를 반환.

In [20]:
# 벡터 저장소에서 "테슬라 창업자는 누구인가요?" 와 유사한 문서 청크를 검색해봄.
query = "테슬라 창업자는 누구인가요?"
result_docs = vectorstore.similarity_search(
    query,
    k=4  # 가장 유사한 문서 4개를 가져옴 (기본값)
)

print(f"검색 결과의 길이: {len(result_docs)}")
print(f"검색 결과의 첫 번째 문서: {result_docs[0].page_content[:100]}...")
# for doc in result_docs: # 검색된 모든 문서를 보고 싶다면 주석 해제
#     print("--- 검색된 문서 ---")
#     print(doc.page_content)
#     print(f"(출처: {doc.metadata['source']})")

검색 결과의 길이: 4
검색 결과의 첫 번째 문서: 텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타...


### 1.5 검색기 (Retrievers)

검색기는 벡터 저장소 같은 소스에서 특정 쿼리에 관련된 문서를 가져오는 역할을 하는 LangChain의 표준 인터페이스임.
벡터 저장소 자체도 `similarity_search` 같은 메서드를 제공하지만, 검색기 인터페이스(`as_retriever()`)를 사용하면 LangChain의 다른 컴포넌트(특히 체인)와 더 쉽게 결합할 수 있음.

`as_retriever()`를 사용하면 벡터 저장소를 검색기로 변환할 수 있음.

**주요 파라미터 (`search_kwargs`):**
- `k`: 검색할 문서의 개수를 지정. `similarity_search`의 `k`와 동일한 역할.
- `score_threshold`: 특정 유사도 점수 이상의 문서만 반환하도록 필터링 가능. (예: `0.7`)
- `fetch_k`: 유사도 계산 전에 내부적으로 가져올 문서의 개수. MMR(Maximal Marginal Relevance) 같은 고급 검색 전략 사용 시 의미 있음.

**팁:**
- 단순히 유사도 높은 문서만 가져오는 것 외에도, 다양한 검색 전략을 가진 검색기들이 있음. (예: `MultiQueryRetriever`, `ParentDocumentRetriever`, `SelfQueryRetriever` 등) 상황에 맞게 선택하면 RAG 성능을 더 끌어올릴 수 있음.

In [21]:
# 벡터 저장소를 Retriever로 변환. 
# search_kwargs를 통해 검색 시 옵션(예: 가져올 문서 개수 k)을 설정할 수 있음.
retriever = vectorstore.as_retriever(search_kwargs={"k": 2}) # 상위 2개 문서만 가져오도록 설정

# retriever.invoke() 메서드로 문서 검색 실행
relevant_docs = retriever.invoke(query) # query는 이전 셀에서 정의한 "테슬라 창업자는 누구인가요?"

print(f"검색된 관련 문서 수: {len(relevant_docs)}")
print(f"첫 번째 관련 문서 내용 미리보기: {relevant_docs[0].page_content[:100]}...")

검색된 관련 문서 수: 2
첫 번째 관련 문서 내용 미리보기: 텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타...


### 1.6 언어 모델 (LLMs)

RAG의 'G'(Generation)를 담당하는 핵심 엔진, 바로 대규모 언어 모델(LLM)임.
검색기가 찾아온 관련 문서를 참고자료(Context) 삼아, 사용자 질문에 대한 최종 답변을 생성함.
LangChain은 OpenAI의 GPT 모델들, HuggingFace의 오픈소스 모델 등 다양한 LLM을 지원함.

여기서는 OpenAI의 `gpt-4o-mini` 모델을 사용.

**주요 파라미터:**
- `model` (또는 `model_name`): 사용할 LLM 모델 이름. (예: `gpt-4o-mini`, `gpt-4`, `gpt-3.5-turbo`)
- `temperature`: 생성될 텍스트의 무작위성(창의성) 조절. 0에 가까울수록 결정적이고 일관된 답변, 높을수록 다양하고 창의적인 답변이 나옴. 보통 0.0 ~ 1.0 사이 값을 사용. RAG에서는 사실 기반 답변이 중요하므로 낮게 설정하는 경우가 많음 (예: 0 또는 0.1).
- `max_tokens`: 생성할 최대 토큰 수. 답변 길이를 제어함. 너무 작으면 답변이 중간에 잘릴 수 있고, 크면 비용이 증가할 수 있음.

**LLM 사용 시 주의점 (특히 RAG에서):**
- **환각(Hallucination) 현상:** LLM이 학습 데이터에 없거나 잘못된 정보를 마치 사실인 것처럼 생성하는 현상. RAG는 이 문제를 완화하는 데 큰 도움을 줌.
- **컨텍스트 주입의 중요성:** 검색된 관련 문서를 LLM 프롬프트에 잘 넣어줘야 함. 이것이 RAG의 핵심!

**장점 (OpenAI LLMs):**
- 강력한 언어 이해 및 생성 능력.
- 다양한 크기와 성능의 모델 선택 가능.

**단점 (OpenAI LLMs):**
- API 호출 비용 발생.
- 인터넷 연결 필요.
- 데이터가 OpenAI 서버로 전송됨 (민감 데이터의 경우 고려 필요).

In [22]:
# 1. Context를 제공하지 않고 LLM에게 질문했을 때의 답변 (환각 발생 가능성 테스트)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0,        # 답변의 일관성을 위해 0으로 설정 (창의성 낮춤)
    max_tokens=100,       # 답변 최대 길이를 100 토큰으로 제한
    )

response_without_context = llm.invoke("테슬라 창업자는 누구인가요?") # query는 이전 셀에서 정의
print(f"LLM 응답 (컨텍스트 없음): {response_without_context}")
# 위 응답을 보면, 우리 데이터에는 없는 '엘론 머스크'를 언급하며 일반적인 웹 지식을 바탕으로 답하거나, 
# 혹은 정보가 부족하면 부정확한 답변을 할 수 있음. max_tokens 때문에 답변이 잘리기도 함.

LLM 응답 (컨텍스트 없음): content='테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하여 회사를 이끌어왔습니다. 따라서 엘론 머스크는 테슬라의 가장 잘' response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 17, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_54eb4bd693', 'finish_reason': 'length', 'logprobs': None} id='run-1d3e94a9-1bc2-4f85-a040-d8687507ff5a-0' usage_metadata={'input_tokens': 17, 'output_tokens': 100, 'total_tokens': 117}


**컨텍스트(Context)를 활용한 답변 생성:**

이제 RAG의 핵심! 검색된 관련 문서(`relevant_docs`)를 LLM에게 참고자료로 함께 제공하여 질문에 답변하도록 해보겠음.
이렇게 하면 LLM은 주어진 컨텍스트 내에서 정보를 찾아 답변하므로, 훨씬 정확하고 신뢰도 높은 결과를 얻을 수 있음.

In [23]:
# 2. Context를 제공하고 LLM에게 질문했을 때의 답변

# 검색된 문서들 중 첫 번째 문서의 내용을 컨텍스트로 사용
# 실제로는 여러 문서를 조합하거나, 더 정교한 프롬프팅 기법을 사용함
context_for_llm = relevant_docs[0].page_content 

# LLM에게 전달할 프롬프트를 구성. 
# 컨텍스트를 명확히 제시하고, 해당 컨텍스트를 기반으로 답변하도록 지시하는 것이 중요.
query_with_context = f"""다음 컨텍스트를 주의 깊게 읽고 질문에 답변해주세요:
컨텍스트:
----------
{context_for_llm}
----------

질문: {query}

답변: 
"""
print(query_with_context) # LLM에게 전달될 최종 프롬프트 확인

다음 컨텍스트를 주의 깊게 읽고 질문에 답변해주세요:
컨텍스트:
----------
텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 머스크(Elon Musk)의 적극적인 참여를 받았습니다. 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.
----------

질문: 테슬라 창업자는 누구인가요?

답변: 



In [24]:
response_with_context = llm.invoke(query_with_context)
print(f"LLM 응답 (컨텍스트 활용): {response_with_context.content}")
# 컨텍스트를 제공하니, 우리 데이터에 있는 '마틴 에버하드와 마크 타페닝'을 정확히 찾아 답변함!

LLM 응답 (컨텍스트 활용): 테슬라의 창립자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다.


### 1.7 전체 RAG 파이프라인 구성 (Chains)

지금까지 개별 컴포넌트(로더, 스플리터, 임베딩, 벡터 저장소, 리트리버, LLM)들을 하나씩 살펴봤음.
LangChain의 진가는 이런 컴포넌트들을 '체인(Chain)'으로 엮어 하나의 파이프라인으로 자동화할 수 있다는 점임.
LangChain Expression Language (LCEL)을 사용하면 파이프라인을 더 직관적이고 유연하게 구성할 수 있지만, 여기서는 기본적인 체인 구성 함수들을 사용해봄.

**주요 체인 구성 요소:**
- **프롬프트 템플릿 (`ChatPromptTemplate`):** LLM에게 전달할 프롬프트의 틀을 정의함. 컨텍스트(`{context}`)와 사용자 질문(`{input}`) 같은 변수를 포함할 수 있음.
  - `hub.pull()`: LangChain Hub에 공유된 검증된 프롬프트 템플릿을 쉽게 가져와 사용할 수 있음. (옵션1)
  - `ChatPromptTemplate.from_template()`: 직접 프롬프트 문자열을 작성하여 템플릿을 만듦. (옵션2)
- **`create_stuff_documents_chain`:** 검색된 여러 문서(청크)들을 하나의 컨텍스트로 "stuffing"(채워넣기)하여 LLM 프롬프트에 전달하고, LLM으로부터 답변을 받는 체인을 생성함. 가장 간단한 문서 결합 방식.
  - **장점:** 구현이 쉽고, 모든 검색된 정보를 LLM이 한 번에 볼 수 있음.
  - **단점:** 검색된 문서들의 총 길이가 LLM의 컨텍스트 윈도우 제한을 넘으면 에러 발생. 문서가 많을수록 비용과 처리 시간 증가.
  - (다른 문서 결합 방식: `map_reduce`, `refine`, `map_rerank` 등. 문서가 매우 많거나 길 때 고려)
- **`create_retrieval_chain`:** 사용자 질문을 받아 Retriever로 관련 문서를 검색하고, 검색된 문서를 `create_stuff_documents_chain` 같은 문서 결합 체인에 전달하여 최종 답변을 생성하는 전체 RAG 흐름을 만드는 체인임.

**흐름 요약:**
1.  사용자 질문 (`input`)이 들어옴.
2.  `create_retrieval_chain`은 이 `input`을 Retriever에게 전달.
3.  Retriever는 유사한 문서들을 검색하여 `context`로 만듦.
4.  `input`과 `context`는 `combine_docs_chain` (여기서는 `create_stuff_documents_chain`으로 생성)으로 전달됨.
5.  `combine_docs_chain`은 프롬프트 템플릿에 `input`과 `context`를 채워 넣어 LLM을 호출.
6.  LLM이 생성한 답변(`answer`)을 최종 결과로 반환 (이때 `input`과 `context`도 함께 반환됨).

In [25]:
from langchain import hub # LangChain Hub에서 프롬프트 등을 가져올 때 사용
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain # Retriever와 문서결합 체인을 연결
from langchain.chains.combine_documents import create_stuff_documents_chain # 문서를 "stuff" 방식으로 결합

# LLM 다시 초기화 (이전 셀에서 이미 했지만, 명시적으로)
llm_for_chain = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- 프롬프트 설정 --- #
# 옵션 1: LangChain Hub에서 검증된 프롬프트 가져오기 (인터넷 연결 필요)
# 이 프롬프트는 RAG 상황에 맞게 잘 디자인되어 있음.
# retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

# 옵션 2: 직접 프롬프트 템플릿 작성하기
# {context} 에는 검색된 문서 내용이, {input} 에는 사용자 질문이 들어감.
retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
당신은 친절한 AI 어시스턴트입니다. 다음 컨텍스트 정보를 바탕으로 사용자의 질문에 답변해주세요. 
컨텍스트 정보가 질문과 관련이 없다면, "주어진 정보로는 답변할 수 없습니다."라고 솔직하게 말해주세요.

컨텍스트:
{context}

질문: {input}

답변:
""")

# --- 체인 생성 --- #
# 1. 문서 결합 및 LLM 호출 체인 생성 (Stuffing 방식)
# 검색된 문서들(context)과 사용자 질문(input)을 프롬프트에 넣어 LLM을 호출함.
combine_docs_chain = create_stuff_documents_chain(
    llm=llm_for_chain, 
    prompt=retrieval_qa_chat_prompt
)

# 2. 검색(Retrieval) 체인 생성
# Retriever (앞에서 만든 vectorstore.as_retriever())와 위에서 만든 combine_docs_chain을 연결.
# 이 체인은 먼저 retriever로 문서를 가져오고, 그 결과를 combine_docs_chain에 전달함.
rag_chain = create_retrieval_chain(
    retriever=retriever, # relevant_docs = retriever.invoke(query) 에서 사용한 그 retriever
    combine_docs_chain=combine_docs_chain
)

# --- 체인 실행 --- #
query_for_chain = "테슬라 창업자는 누구인가요?" # query 변수 재사용
# rag_chain.invoke()의 입력은 딕셔너리 형태여야 하며, 프롬프트에 정의된 'input' 키를 사용함.
response_from_chain = rag_chain.invoke({"input": query_for_chain})

In [26]:
# 체인 실행 결과 출력
# 결과는 딕셔너리 형태로, 'input', 'context', 'answer' 키를 포함함.
# 'context'에는 retriever가 검색한 문서(Document 객체 리스트)가 들어있음.
# 'answer'에는 LLM이 생성한 최종 답변이 들어있음.
print(response_from_chain)

{'input': '테슬라 창업자는 누구인가요?', 'context': [Document(metadata={'source': './data\\Tesla_KR.txt'}, page_content='텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 머스크(Elon Musk)의 적극적인 참여를 받았습니다. 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.'), Document(metadata={'source': './data\\Tesla_KR.txt'}, page_content='2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년 대비 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 누적 글로벌 판매량은 4,962,975대를 넘어섰습니다. SMT 패키징(SMT Packaging)에 따르면, 테슬라의 2023년 판매량은 글로벌 전기 자동차 시장의 약 12.9%를 차지했습니다.')], 'answer': '테슬라의 창립자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다.'}


In [27]:
# 응답 객체의 키 확인
response_from_chain.keys()

dict_keys(['input', 'context', 'answer'])

In [28]:
# 최종 답변만 보고 싶다면 'answer' 키로 접근
response_from_chain["answer"]

'테슬라의 창립자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다.'

**`[추가 설명] create_stuff_documents_chain` 자세히 보기**

`create_stuff_documents_chain`은 이름처럼 검색된 문서들을 'stuff'(채워넣기) 방식으로 처리하는 체인임.
얘는 독립적으로 사용될 수도 있음. 즉, Retriever 없이 미리 준비된 문서들을 컨텍스트로 제공하고 질문에 답변하게 할 수 있음.

**작동 방식:**
1.  입력으로 `input`(사용자 질문)과 `context`(참고 문서 리스트)를 받음.
2.  이 `input`과 `context`를 지정된 프롬프트 템플릿에 채워 넣음.
3.  완성된 프롬프트를 LLM에게 전달하여 답변을 생성함.
4.  LLM의 답변을 반환함 (이때는 `answer` 키가 아니라 바로 답변 문자열이 나옴).

In [29]:
from langchain_core.documents import Document # Document 객체를 직접 만들 때 필요

# 위에서 만든 combine_docs_chain (create_stuff_documents_chain의 결과물)을 재사용
# stuff_chain = create_stuff_documents_chain(llm_for_chain, retrieval_qa_chat_prompt) # 동일함

# 참고할 문서를 직접 Document 객체 리스트로 만들어줌
manual_documents = [
    Document(page_content="LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다."),
    Document(page_content="LangChain은 파이썬과 자바스크립트를 지원합니다.")
]

response_from_stuff_chain = combine_docs_chain.invoke({
    "input": "LangChain은 어떤 언어를 지원하나요?",
    "context": manual_documents # Retriever 대신 직접 문서를 context로 제공
})

# create_stuff_documents_chain의 직접적인 출력은 LLM의 답변 문자열임.
print(response_from_stuff_chain)

LangChain은 파이썬과 자바스크립트를 지원합니다.


**`[추가 설명] create_retrieval_chain` 자세히 보기**

`create_retrieval_chain`은 두 개의 주요 컴포넌트 체인을 결합함:
1.  **Retriever:** 사용자 질문(`input`)을 받아 관련 문서를 검색하는 역할.
2.  **Document Combining Chain (예: `combine_docs_chain`):** 검색된 문서(`context`)와 사용자 질문(`input`)을 받아 최종 답변(`answer`)을 생성하는 역할.

이 함수는 이 두 단계를 자동으로 연결하여 전체 RAG 파이프라인을 쉽게 만들 수 있도록 도와줌.
Retriever 부분에는 `vectorstore.as_retriever()` 뿐만 아니라, 다른 종류의 Retriever 객체도 전달할 수 있음.

아래 코드는 `rag_chain`을 생성할 때 `retriever` 변수 대신 `vectorstore.as_retriever()`를 직접 사용한 예시임. 
(결과는 위에서 `rag_chain`을 실행한 것과 동일함. `retriever`가 `vectorstore.as_retriever(search_kwargs={"k": 2})`로 이미 정의되어 있었기 때문. 만약 `search_kwargs`를 다르게 하고 싶다면 여기서 새로 지정 가능)

In [30]:
# create_retrieval_chain에 retriever를 직접 생성해서 전달하는 예시
# vectorstore.as_retriever()를 호출할 때 search_kwargs를 다르게 설정하면 다른 검색 결과를 얻을 수 있음.
# 여기서는 기본 k=4를 사용하도록 retriever를 새로 만듦.
alternative_retrieval_chain = create_retrieval_chain(
    retriever=vectorstore.as_retriever(), # k의 기본값인 4개 문서를 가져옴
    combine_docs_chain=combine_docs_chain # 위에서 만든 stuff 체인 재사용
)

# 체인 실행
alternative_response = alternative_retrieval_chain.invoke({"input": query_for_chain})

# 결과 출력 (context에 4개의 문서가 포함된 것을 확인할 수 있음)
print(alternative_response)

{'input': '테슬라 창업자는 누구인가요?', 'context': [Document(metadata={'source': './data\\Tesla_KR.txt'}, page_content='텍사스주 오스틴에 본사를 둔 테슬라(Tesla, Inc.)는 미국의 대표적인 전기 자동차 제조업체입니다. 2003년 마틴 에버하드(Martin Eberhard, CEO)와 마크 타페닝(Marc Tarpenning, CFO)이 설립한 테슬라는 2004년 일론 머스크(Elon Musk)의 적극적인 참여를 받았습니다. 페이팔(PayPal)과 짚투(Zip2)의 공동 창립자인 머스크는 최대 주주이자 회장이 되어 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 저명한 물리학자이자 전기 공학자인 니콜라 테슬라(Nikola Tesla)의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.'), Document(metadata={'source': './data\\Tesla_KR.txt'}, page_content='2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년 대비 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 누적 글로벌 판매량은 4,962,975대를 넘어섰습니다. SMT 패키징(SMT Packaging)에 따르면, 테슬라의 2023년 판매량은 글로벌 전기 자동차 시장의 약 12.9%를 차지했습니다.'), Document(metadata={'source': './data\\Rivian_KR.txt'}, page_content='2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다.'), Document(metadata={'s